In [2]:
pip install z3-solver


     ---------------------------------------- 58.9/58.9 MB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [11]:
from z3 import *

def read_input(file_name):
    with open(file_name, 'r') as file:
        n_vertices, n_interference_edges, n_affinity_edges, K1, K2 = map(int, file.readline().split())
        interference_edges = [tuple(map(int, file.readline().split())) for _ in range(n_interference_edges)]
        affinity_edges = [tuple(map(int, file.readline().split())) for _ in range(n_affinity_edges)]
    return n_vertices, interference_edges, affinity_edges, K1, K2


In [12]:
def solve_graph_coloring(n_vertices, interference_edges, affinity_edges, K1, K2):
    solver = Solver()
    colors = [Int(f'color_{i}') for i in range(1, n_vertices + 1)]
    for color in colors:
        solver.add(1 <= color, color <= K1)
    for u, v in interference_edges:
        solver.add(colors[u - 1] != colors[v - 1])
    affinity_constraints = [colors[x - 1] == colors[y - 1] for x, y in affinity_edges]
    solver.add(Sum([If(c, 1, 0) for c in affinity_constraints]) >= K2)
    if solver.check() == sat:
        model = solver.model()
        solution = [model.evaluate(color) for color in colors]
        return 'Yes', solution
    else:
        return 'No', []

In [13]:
def write_output(file_name, result, solution):
    with open(file_name, 'w') as file:
        file.write(result + '\n')
        if result == 'Yes':
            for color in solution:
                file.write(str(color) + '\n')
input_file = 'sample_5_3.txt'
output_file = 'output_5_3.txt'

n_vertices, interference_edges, affinity_edges, K1, K2 = read_input(input_file)
result, solution = solve_graph_coloring(n_vertices, interference_edges, affinity_edges, K1, K2)
write_output(output_file, result, solution)
